In [65]:
import numpy as np
import scipy

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score

#Training and Visualizaing a Decision Tree

In [2]:
iris = load_iris()
x = iris.data[:, 2:] # petal lenght and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(x, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [3]:
export_graphviz(
    tree_clf,
    out_file='iris_tree.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

In [4]:
# Convert the dot image into a png

!dot -Tpng iris_tree.dot -o iris_tree.png

# Making Predictions

The Gini attribute measures the impurity of each node. If all the instances it applies to belong to the same class, the node is pure and therefore $\text{gini}=0$

- Gini impurity
> $G_{i} = 1 - \sum\limits_{k-1}^{n}{p_{i,k}^2}$
> - $p_{i, k}$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node

# Estimating Class Probabilities

In [5]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [6]:
tree_clf.predict([[5, 1.5]])

array([1])

# The CART Training Algorithm

Scikit-Learn uses the CART (*Classification and Regression Tree*) algorithm to train the Decissions Trees. This algorithm splits the training data set in 2 subsets selecting a using a feature $k$ and a threshold $t_{k}$ such that the pair $(k, t_{k})$ produces the purest subsets.

- CART cost function for classification
> $J(k, t_{k}) = \frac{m_{\text{left}}}{m}G_{\text{left}} + \frac{m_{\text{right}}}{m}G_{\text{right}}$

# Giny Impurity or Entropy?

The Entropy in a node is zero when all the instances belong to the same class. 

- Entropy
> $H_{i} = \underbrace{-\sum\limits_{k=1}^{n}{p_{i,k}\text{log}_{2}(p_{i,k})}}_{p_{i,k}\neq \text{ 0}}$

#Regression

In [7]:
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(x, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [8]:
export_graphviz(
    tree_reg,
    out_file='regresion_tree.dot',
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

The CART algorithm now does not try to reduce the impurity gini, instead it tries to minimize the cost function MSE for regression.

- CART cost function for regression
> $J(k, t_{k}) = \frac{m_{\text{left}}}{m}MSE_{\text{left}} + \frac{m_{\text{right}}}{m}MSE_{\text{right}}$
> $\text{where  }\begin{cases} {MSE_{\text{node}} = \sum\limits_{i\epsilon\text{node}}{(\hat{y}-y^{(i)})^2}} \\ { \hat{y}_{\text{node}} = \frac{1}{m_{\text{node}}} \sum\limits_{i\epsilon\text{node}}{y^{(i)}}}\end{cases}$

#Exercises

1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?

> If the training set is used without restrictions, the decission tree will create a leaf node for every instance to overfit the training data. In this case since the prediction complexity of the algorithm is $O(\text{log}_2 (m))$ the result would be a depth of $\approx 20$.

2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?

> The gini's impurity of a node is generally lower than its parent's, and this is due to the CART algorithm that is a greedy algoritm. However, it may happpen that the gini's impurity may be higher in a son node than in its parent's. This may be caused by the classes proportions that may vary when splitting the data on the previous node.

3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?

> Yes, decreasing the ```max_depth``` hyperparameter will regularize the model and therefore will reduce the overfitting, making the model more general. 




4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?

> No, decision tree algorithm is not sensible to the scaling of the features. However decreasing the regularization by modifying the hyperparameters may solve this underfitting.

5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

> The computational complexity of training a Decision Tree is O(n × m log(m)). So if you multiply the training set size by 10, the training time will be multiplied by K = (n × 10m × log(10m)) / (n × m × log(m)) = 10 × log(10m) / log(m). If m = 106 , then K ≈ 11.7, so you can expect the training time to be roughly 11.7 hours.

6. If your training set contains 100,000 instances, will setting presort=True speed up training?

> Presorting the training set only speeds up the training if the instances are less than a few thousands. In this case presorting will slow down the training time.

7. Train and fine-tune a Decision Tree for the moons dataset by following these
steps:
- Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.
- Use train_test_split() to split the dataset into a training set and a test set.
- Use grid search with cross-validation (with the help of the GridSearchCV
class) to find good hyperparameter values for a DecisionTreeClassifier.
Hint: try various values for max_leaf_nodes.
- Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to 87%
accuracy.


In [9]:
x, y = make_moons(n_samples=100000, noise=0.4)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [10]:
x_train

array([[-0.6425703 ,  0.78767354],
       [ 1.04341145, -0.44044197],
       [ 0.08121305,  1.01189925],
       ...,
       [-0.31574628,  0.86578331],
       [ 0.71354244,  1.07212082],
       [ 0.75734444,  0.96778559]])

In [11]:
tree_clf_moons = DecisionTreeClassifier()

tree_params = [
    {'max_depth': [3, 4, 5, 6, 7], 'criterion':['gini', 'entropy'], 'min_samples_split': [10, 100, 1000], 'max_leaf_nodes': [10, 25, 50, 100, 150, 250]}
]

gridcv_tree_moons = GridSearchCV(tree_clf_moons, tree_params, cv=5, scoring='accuracy', return_train_score=True)

In [12]:
gridcv_tree_moons.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['gini', 'entropy'],
                  

In [13]:
gridcv_tree_moons.best_params_

{'criterion': 'gini',
 'max_depth': 7,
 'max_leaf_nodes': 50,
 'min_samples_split': 10}

In [14]:
optimized_tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=7, max_leaf_nodes=50, min_samples_split=10)

optimized_tree_clf.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=50,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
def model_evaluation_clf(model, x_test, y_test):

  name_model = type(model).__name__

  y_hat = model.predict(x_test)

  accuracy_model = round(accuracy_score(y_test, y_hat), 4)
  precision_model = round(precision_score(y_test, y_hat), 4)
  recall_model = round(recall_score(y_test, y_hat), 4)
  f1_model = round(f1_score(y_test, y_hat), 4)
  conf_matrix_model = confusion_matrix(y_test, y_hat)

  print(f'These are the results of the model {name_model}:')
  print(f'\tAccuracy: {accuracy_model}')
  print(f'\tPrecision: {precision_model}')
  print(f'\tRecall: {recall_model}')
  print(f'\tF1: {f1_model}')
  print(f'\tConfusion Matrix:\n\t{conf_matrix_model}')

In [16]:
model_evaluation_clf(optimized_tree_clf, x_test, y_test)

These are the results of the model DecisionTreeClassifier:
	Accuracy: 0.8568
	Precision: 0.8544
	Recall: 0.8609
	F1: 0.8576
	Confusion Matrix:
	[[8509 1470]
 [1394 8627]]


8. Grow a forest by following these steps:
- Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this.
- Train one Decision Tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 Decision Trees on the test
set. Since they were trained on smaller sets, these Decision Trees will likely
perform worse than the first Decision Tree, achieving only about 80%
accuracy.
- Now comes the magic. For each test set instance, generate the predictions of
the 1,000 Decision Trees, and keep only the most frequent prediction (you can
use SciPy’s mode() function for this). This approach gives you majority-vote
predictions over the test set.
- Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations,
you have trained a Random Forest classifier!

In [17]:
rs_trees = ShuffleSplit(n_splits=1000)

In [53]:
y_test.shape

(20000,)

In [94]:
forest_tree = DecisionTreeClassifier(criterion='gini', max_depth=7, max_leaf_nodes=50, min_samples_split=10)

predictions = np.empty((1000,y_test.shape[0]))

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

i = 0

for train_idx, test_idx in rs_trees.split(x_train):

  x_train_split = x_train[train_idx]
  y_train_split = y_train[train_idx]
  x_test_split = x_train[test_idx]
  y_test_split = y_train[test_idx]

  forest_tree.fit(x_train_split, y_train_split)

  y_hat = forest_tree.predict(x_test)

  accuracy_scores.append(accuracy_score(y_test, y_hat))
  precision_scores.append(precision_score(y_test, y_hat))
  recall_scores.append(recall_score(y_test, y_hat))
  f1_scores.append(f1_score(y_test, y_hat))

  predictions[i] = y_hat

  i += 1

In [95]:
# Get the mean of the metrics for the 1000 predictions made by the 1000 decision trees

print('This are the metrics for the decision trees:')
print(f'\tAccuracy: {round(np.mean(accuracy_scores), 4)}')
print(f'\tPrecision: {round(np.mean(precision_scores), 4)}')
print(f'\tRecall: {round(np.mean(recall_scores), 4)}')
print(f'\tF1: {round(np.mean(f1_scores), 4)}')

This are the metrics for the decision trees:
	Accuracy: 0.8579
	Precision: 0.8556
	Recall: 0.8619
	F1: 0.8587


In [96]:
# Get the prediction for every test instance by getting the mode of all the predictions, and then flatten the transpose of the result
forest_predictions = scipy.stats.mode(predictions)[0]
forest_predictions = forest_predictions.T.flatten()

# Get the metrics for this predictions that is the ensemble of the 1000 decision trees
forest_accuracy = accuracy_score(y_test, forest_predictions)
forest_precision = precision_score(y_test, forest_predictions)
forest_recall = recall_score(y_test, forest_predictions)

print(f'The metrics of our forest is:')
print(f'\tAccuracy: {forest_accuracy}')
print(f'\tPrecision: {forest_precision}')
print(f'\tRecall: {forest_recall}')

The metrics of our forest is:
	Accuracy: 0.85765
	Precision: 0.8543065981825365
	Recall: 0.8630875162159465
